In [1]:
import os
os.chdir(r"C:\Users\Administrator\OneDrive\课件\自然语言处理")
seg_dict = []
with open('seg_dict.txt', encoding='utf-8') as f:
    seg_dict.append([n.split() for n in f.readlines()])
dic=[n[0] for n in seg_dict[0] if n[2]!='nr'] #去除字典中原先有的所有人名
with open('金庸-侠客行txt全本精校版.txt', encoding='utf-8') as f:
    content = ' '.join([n for n in f.readlines()])

In [2]:
def count_name(name, text):
    '''
    该函数在全文中统计输入名字出现的次数
    '''
    count = 0
    while len(text)>0:
        a = text.find(name)
        if a == -1:
            break
        else:
            count +=1
            text = text[a+len(name):]
    return count
def name_score(name, text, loc,dic):
    '''
    该函数用于计算一个名字的评分
    '''
    times = count_name(name, text)
    #若包含姓的前后3个字若有一个词语组合存在与字典中，就认为不是人名，分值为-1，例如"正在东张西望的"，在识别张西望是否为名字时，因为东张西望在词典中，故认为张西望不是人名
    if name.find(' ')!=-1 or text[loc-3:loc+1] in dic or text[loc-2:loc+2] in dic or text[loc-1:loc+3] in dic or text[loc-2:loc+1] in dic or text[loc-1:loc+2] in dic or text[loc-1:loc+1] in dic or text[loc:loc+2] in dic or text[loc:loc+3] in dic or text[loc:loc+4] in dic:
        score = -1
    #若除了姓氏外，名字和名字后面字符组合在字典中，则给予扣分
    elif text[loc+len(name)-1:loc+len(name)+1] in dic or text[loc+len(name)-1:loc+len(name)+2] in dic or text[loc+len(name)-1:loc+len(name)+3] in dic:
        score = times-4
    #考虑到会有姓氏加词语的叫法，例如张师傅，该种情况酌情扣分
    elif text[loc+len(name)-2:loc+len(name)] in dic:
        score = times-2
    else:
        score = times
    return score

In [3]:
from tqdm import tqdm
def find_name(text,dic_):
    dic_ = dic
    name = []
    impossible_name = []
    for n in [',','.',';',':','，','。','；','：','‘','’','“','”','!','！','?','？','、','》','《','还','已','而','向','刺','姓','吧','虽','者','也','死','上','名','下','前','着','她','就','是','哼','…','了','哦','噢','这','那','呐','的','你','我','脸']:
        text = ' '.join(text.split(n))
    #print(text)
    try:
        with tqdm(range(len(text))) as t:
            for i in t:
                '''
                以下开始遍历全文 若文字在姓氏中，进入计算分值和比较程序，每识别一个名字就将其加到字典中
                '''
                if text[i] in last_names:
                    t.set_description("正在识别人名：{}".format(text[i]))
                    t.update(1)
                    content = text[:]
                    loc = i
                    #取两个字或三个字作为姓名
                    name_2 = content[loc:loc+2]
                    name_3 = content[loc:loc+3]
                    if name_2 not in name and name_3 not in name:  #若没有出现过该名字
                        score_name_2 = name_score(name_2, content[:], loc, dic_)
                        score_name_3 = name_score(name_3, content[:], loc, dic_)*1.1  #三个字名字适当甲权
                        if score_name_3>5:       #若三个字名字分数高于5，则加倍算分（防止类似 丁不三 和 丁不 丁不三分值低于丁不的情况）
                            score_name_3 = score_name_3 * 5
                        if score_name_2>20:      #两个字名字也适当加权
                            score_name_2 = score_name_2 * 1.5
                        #print(name_2,score_name_2,name_3,score_name_3)
                        if score_name_2<=0 and score_name_3<=0:
                            impossible_name.append(name_2)
                            impossible_name.append(name_3)
                            continue
                        #当其中一个名字在不可能名字序列时，另一个名字又大于某特定值，认定该名字为人名，并将另一个名字加到不可能列表
                        elif score_name_2>2 and name_3 in impossible_name and name_2 not in impossible_name:
                            name.append(name_2)
                            dic_.append(name_2)
                            if name_3 not in impossible_name:
                                impossible_name.append(name_3)
                        elif score_name_3>2 and name_2 in impossible_name and name_3 not in impossible_name:
                            name.append(name_3)
                            dic_.append(name_3)
                            if name_2 not in impossible_name:
                                impossible_name.append(name_2)
                        #认为分值大并且分值大于3且不在不可能列表中的名字为人名，并将另一个名字加到不可能列表
                        elif score_name_2>score_name_3 and score_name_2 >3 and name_2 not in impossible_name:
                            name.append(name_2)
                            dic_.append(name_2)
                            if name_3 not in impossible_name:
                                impossible_name.append(name_3)
                        elif score_name_2<=score_name_3 and score_name_3 >3 and name_3 not in impossible_name:
                            name.append(name_3)
                            dic_.append(name_3)
                            if name_2 not in impossible_name:
                                impossible_name.append(name_2)
    except KeyboardInterrupt:
        t.close()
        raise
    t.close()
    return name


In [4]:
#查找姓氏
last_names = ['李','周','阖','吴','王','石','丁','史','白','吕','陈','安','梅','温','谢','范','冯','司','邱','褚','廖','梁','耿','张','齐','贝','郑','米','风','天','展','尤','花','闵','杨','龙','封','侍','汪']

In [5]:
dic=[n[0] for n in seg_dict[0] if n[2]!='nr'] #去除字典中原先有的所有人名
name = find_name(content, dic)
print(name)

正在识别人名：石: 100%|██████████████████████████████████████████████████████████████| 389623/389623 [03:57<00:00, 1643.38it/s]


['李白', '王老', '安寨主', '吴道通', '周牧', '石庄主', '石清', '闵柔', '石氏', '石夫人', '周世兄', '安某', '安奉日', '冯振武', '耿师', '耿万钟', '谢烟客', '王万仞', '谢先生', '王师哥', '梅女侠', '花万紫', '花师妹', '谢某', '石大哥', '石中玉', '封师兄', '封万里', '白老爷', '石大嫂', '白师哥', '白万剑', '耿兄', '白师嫂', '王师弟', '褚师弟', '丁不三', '丁不二', '丁不四', '丁氏', '褚万春', '封师哥', '白自在', '丁丁', '司徒帮', '司徒横', '米香主', '贝大夫', '贝海石', '贝先生', '石破天', '天哥', '米横野', '侍剑', '展飞', '邱山风', '陈冲之', '陈香主', '花姑娘', '丁珰', '石郎', '丁老', '丁三爷', '丁家', '白师傅', '白某', '廖师叔', '汪万翼', '白师兄', '丁不叫', '史小翠', '史婆婆', '张三', '史亿刀', '丁四', '李四', '尤得胜', '天虚', '石师弟', '闵师妹', '白爷爷', '杨光', '范一飞', '风良', '丁不闪', '吕正平', '石恩公', '司徒大', '丁姑娘', '白大', '杨老', '白老伯', '白师伯', '齐师', '廖师弟', '梁师', '梁四', '廖自砺', '齐自勉', '梁自进', '龙岛主', '龙木', '郑光芝', '丁先生', '梅芳姑', '梅女子', '龙鞭法', '温三', '温仁厚', '阖庐', '梅文馨', '石老弟']
